In [1]:
import joblib
import pandas as pd
import numpy as np
import sqlalchemy
import re
import tqdm
from preprocessing import Players, Seasons
from features import Features

pd.set_option('max_columns', None)

db_name = 'postgres'
db_hostname = 'localhost'
db_user = 'postgres'
db_password = 'tttBBB777'
db_table = 'players_list'

engine = sqlalchemy.create_engine(f'postgresql+psycopg2://{db_user}:{db_password}@{db_hostname}/{db_name}')

In [5]:
def __init__(self):
    pass
Features.__init__ = __init__


class Model:
    def __init__(self, model, features):
        self.model = model
        self.features = features

    @classmethod
    def load(cls, path='source'):
        model = joblib.load(f'{path}/model.pkl')
        features = joblib.load(f'{path}/features.pkl')
        return cls(model, features)
    
    def predict(self, player_id, postseason):
        df = self.build_features(player_id, postseason).tail(1)
        return self.model.predict(df[self.features])
        
    def build_features(self, player_id, postseason):
        f = Features()
        df = self.get_player_stats(player_id, postseason)
        new_season = self.add_new_season(df)
        f.merged = df.append(new_season)
        f.merged = f.merged[[x for x in f.merged if '_lag' not in x]]
        f.merged.drop([f'{x}_{y}' for x in f.aggregates for y in f.stats_columns], axis=1, inplace=True)
        f.merged['draft_entry'] = f.merged['draft_entry'].astype(float)
        f.get_age_at_season()
        f.get_stats_per_game()
        f.get_lags()
        f.get_aggregates()
        return f.merged
    
    def get_player_stats(self, player_id, postseason=0):
        df = pd.read_sql(f"select * from features where player_id = '{player_id}' and postseason_flag = {postseason}", engine)
        df = df.sort_values('season')
        return df
    
    def add_new_season(self, df):
        new_season = df.tail(1).copy()
        new_season['season'] = new_season['season'].map(lambda x: str(int(x.split('-')[0])+1)+'-'+str(int(x.split('-')[1])+1))
        new_season['year'] += pd.offsets.DateOffset(years=1)
        return df

In [6]:
m = Model.load()

In [17]:
df = m.predict('f1ee39c5-ba84-4324-b5ce-81465d4fb76e', 0)

100%|███████████████████████████████████████████| 4/4 [00:00<00:00, 285.66it/s]


In [19]:
df

array([7.41934666])